In [90]:
# Dataset : https://www.kaggle.com/datasets/gauravduttakiit/ants-bees?resource=download
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader

In [91]:
transforms={'train':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),
    'val':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [92]:
data_dir='data/archive/hymenoptera_data'
#image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),transform=transforms[x]) for x in ['train','val']}


In [93]:
image_datasets={}
image_datasets['train']=datasets.ImageFolder(os.path.join(data_dir,'train'),transform=transforms['train'])
image_datasets['val']=datasets.ImageFolder(os.path.join(data_dir,'val'),transform=transforms['val'])

In [94]:
data_loader={}
data_loader['train']=DataLoader(image_datasets['train'],batch_size=32,shuffle=True)
data_loader['val']=DataLoader(image_datasets['val'],batch_size=32,shuffle=True)

In [98]:
model=models.resnet18(pretrained=True)
#print(model)
model_ftrs=model.fc.in_features
model.ft=nn.Sequential()
#model.fc = nn.Sequential()
model.ft.ft1=nn.Linear(model_ftrs,out_features=256)
model.ft.ft2=nn.Linear(256,out_features=2)
#model.ft.ft3=nn.Linear(64,out_features=16)
#model.ft.ft4=nn.Linear(16,out_features=2)
model.fc = model.ft
criterian=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.fc.parameters(),lr=0.0001)


f:\22EG107B03\DL-Lab\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
f:\22EG107B03\DL-Lab\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [99]:
for epoch in range(10):
    for input,label in data_loader['train']:
        output=model(input)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(loss.item())


0.6312088370323181
0.5395984053611755
0.4713052213191986
0.5140424966812134
0.3139335811138153
0.40761375427246094
0.4627936780452728
0.4825107157230377
0.4135815501213074
0.24232104420661926


In [101]:
model.eval()
with torch.no_grad():
    correct=0
    total=0
    for input,label in data_loader['val']:
        output=model(input)
        _,predicted=torch.max(output.data,1)
        total+=label.size(0)
        correct+=(predicted==label).sum().item()
    print('Accuracy: {}%'.format(100*correct/total))
    print(total, correct)

Accuracy: 88.23529411764706%
153 135
